In [1]:
from prisma import Client
import requests
from dotenv import load_dotenv
import os

import json
import sys

import pandas as pd

from collections import namedtuple

current_path = sys.path[0]
sys.path.append(
    current_path[: current_path.find("defi-measurement")]
    + "liquidity-distribution-history"
)

from pool_state import v3Pool
import pandera as pa
from pandera.typing import Series, DataFrame

from datetime import datetime

# Import Azure blob storage SDK
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient


class SwapSchema(pa.DataFrameModel):
    block_number: Series[int]
    transaction_index: Series[int]
    log_index: Series[int]
    amount0: Series[str]
    amount1: Series[str]
    sqrtpricex96: Series[str]
    tick: Series[int]
    tx_hash: Series[str]
    block_ts: Series[datetime]


SimResult = namedtuple('SimResult', ['block_number', 'pool_address', 'n_permutations', 'data_location'])

load_dotenv(override=True)

postgres_uri = os.environ["POSTGRESQL_URI_US"]
azure_blob_connection_string = os.environ["AZURE_STORAGE_CONNECTION_STRING"]

pool_address = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"

In [2]:
db = Client()
await db.connect()

In [3]:
runs = [SimResult(row.block_number, row.pool_address, row.n_permutations, row.data_location) for row in await db.permutationsimulation.find_many(
    where={"n_swaps": {"equals": None}}, # type: ignore
)]

runs[0]

SimResult(block_number=15826457, pool_address='0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', n_permutations=1600, data_location='https://larsb973.blob.core.windows.net/uniswap-pool-pickles/permutation-simulation/simulation_15826457_1600/data.parquet')

In [4]:
from sqlalchemy import create_engine


def swaps_from_pool(pool_address: str, after: int = 0):
    engine = create_engine(postgres_uri)

    df = pd.read_sql(
        f"""
            SELECT block_number, transaction_index,
                log_index, amount0, amount1,
                sqrtpricex96,tick, tx_hash, block_ts
            FROM swaps
            WHERE address = '{pool_address}'
            AND block_number >= {after};
        """,
        engine,
    )

    engine.dispose()
    return df

df = swaps_from_pool(pool_address, int(15e6))

In [5]:
import numpy as np
from tqdm import tqdm


def run_simulation(pool: v3Pool, swaps_parameters: list, pbar=True) -> np.ndarray:
    # Get the sqrtPriceX96 at the start of the block
    sqrtPrice_next = pool.getPriceAt(swaps_parameters[0]["as_of"])

    prices = np.zeros(len(swaps_parameters) + 1, dtype=np.float64)

    for i, s in tqdm(enumerate(swaps_parameters, start=0), disable=not pbar):
        s["givenPrice"] = sqrtPrice_next
        _, heur = pool.swapIn(s, fees=True)
        sqrtPrice_next = heur.sqrtP_next
        prices[i] = 1 / (heur.sqrt_P**2 / 1e12)

    # Calculate the price at the end of the block
    prices[-1] = 1 / (sqrtPrice_next**2 / 1e12)

    return prices


In [6]:
def get_swap_params(pool: v3Pool, swaps: DataFrame[SwapSchema]) -> list:
    # Convert the dataframe into a list of swap parameter dicts
    swaps_parameters = []
    for row in swaps.to_dict(orient="records"):
        # Calculate what is tokenIn and what is tokenOut
        token_in = pool.token0
        amount0 = str(row["amount0"])
        if str(row["amount0"])[0].startswith("-"):
            token_in = pool.token1
            amount0 = str(row["amount1"])

        swapParams = {
            "tokenIn": token_in,
            "input": int(amount0),
            "gasFee": True,
            "as_of": row["block_number"] + 0 / 1e4,
        }

        swaps_parameters.append(swapParams)

    return swaps_parameters

In [7]:

import pickle


def load_pool(
    pool_address: str,
    postgres_uri: str,
) -> v3Pool:
    # Check if pool_cache.pickle exists
    filename = "../cache/pool_cache.pickle"
    if not os.path.exists(filename):
        os.mkdir("cache")
        with open(filename, "wb") as f:
            pickle.dump({}, f)

    # Check if we already have this pool in the cache
    with open(filename, "rb") as f:
        pool_cache = pickle.load(f)
        if pool_address in pool_cache:
            # If it is, load the pool from the cache
            print("Loading pool from cache")
            return pool_cache[pool_address]

    # If it's not in the cache, load it and add it to the cache
    pool = v3Pool(
        poolAdd=pool_address,
        connStr=postgres_uri,
        initialize=False,
        delete_conn=True,
    )

    # Add the pool to the cache
    pool_cache[pool_address] = pool

    # Save the cache
    with open(filename, "wb") as f:
        pickle.dump(pool_cache, f)

    return pool

pool = load_pool(pool_address, postgres_uri)

Loading pool from cache


In [8]:
from pydantic import BaseModel


class Stats(BaseModel):
    baseline: float
    original_std: float
    permutation_stds: list
    mean_permutation_std: float
    permutation_abs_deviations: list
    max_abs_permutation_deviations: list
    mean_max_abs_permutation_deviation: float
    abs_original_deviation: list
    max_abs_original_deviation: float
    permutation_areas: list
    mean_permutation_area: float
    original_area: float


def calculate_stats(
    original_prices: np.ndarray, permutation_prices: np.ndarray
) -> Stats:
    baseline = original_prices[0]
    permutation_abs_deviations = np.abs(permutation_prices - baseline)
    abs_original_deviation = np.abs(original_prices - baseline)

    stats = Stats(
        baseline=baseline.item(),
        original_std=np.std(original_prices).item(),
        permutation_stds=list(np.std(permutation_prices, axis=1)),
        mean_permutation_std=np.mean(np.std(permutation_prices, axis=1)).item(),
        permutation_abs_deviations=list(permutation_abs_deviations.tolist()),
        max_abs_permutation_deviations=list(np.max(permutation_abs_deviations, axis=1)),
        mean_max_abs_permutation_deviation=np.mean(
            np.max(permutation_abs_deviations, axis=1)
        ).item(),
        abs_original_deviation=list(abs_original_deviation),
        max_abs_original_deviation=np.max(abs_original_deviation).item(),
        permutation_areas=list(np.trapz(permutation_abs_deviations, axis=1)),  # type: ignore
        mean_permutation_area=np.mean(np.trapz(permutation_abs_deviations, axis=1)).item(),  # type: ignore
        original_area=np.trapz(abs_original_deviation).item(),  # type: ignore
    )

    return stats

In [15]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import json
from tqdm import tqdm

service_client = BlobServiceClient.from_connection_string(azure_blob_connection_string)
container_name = "uniswap-pool-pickles"

for block_number, pool_address, n_permutations, data_location in tqdm(runs):
    blob_name = data_location.split("uniswap-pool-pickles/")[-1].strip()

    blob_client = service_client.get_blob_client(container_name, blob_name)

    # Retrieve data
    data = pd.read_parquet(data_location).values
    swaps_params = get_swap_params(pool, df[df.block_number == block_number])
    original_prices = run_simulation(pool, swaps_params)

    # Calculate stats
    stats = calculate_stats(original_prices, data)

    new_data = {
        "original_prices": original_prices.tolist(),
        "permutation_prices": data.tolist(),
        "stats": stats.dict(),
    }

    # Save data to a temporary JSON file
    with open("temp.json", "w") as f:
        json.dump(new_data, f)

    filename = f"{blob_name.rsplit('.', 1)[0]}.json".strip()

    new_blob_client = service_client.get_blob_client(container_name, filename)

    with open("temp.json", "rb") as f:
        new_blob_client.upload_blob(f, overwrite=True)


    # Update the database
    await db.permutationsimulation.update_many(
        where={"block_number": block_number, "pool_address": pool_address, "n_permutations": n_permutations},
        data={
            "data_location": new_blob_client.url,
            "n_swaps": len(swaps_params),
            "original_std": stats.original_std,
            "mean_permutation_std": stats.mean_permutation_std,
            "original_area": stats.original_area,
            "mean_permutation_area": stats.mean_permutation_area, 
            "max_abs_original_deviation": stats.max_abs_original_deviation,
            "mean_max_abs_permutation_deviation": stats.mean_max_abs_permutation_deviation,
        },
    )

    # Since all updating was successful, delete the old blob
    blob_client.delete_blob()

13it [00:00, 18.82it/s]00:00<?, ?it/s]
11it [00:00, 17.76it/s]00:01<01:11,  1.19s/it]
11it [00:00, 18.74it/s]00:02<01:03,  1.07s/it]
10it [00:00, 20.16it/s]00:03<00:58,  1.01s/it]
10it [00:00, 17.44it/s]00:04<00:55,  1.03it/s]
9it [00:00, 17.56it/s][00:05<00:56,  1.00s/it]
9it [00:00, 19.49it/s][00:05<00:52,  1.05it/s]
8it [00:00, 18.89it/s][00:06<00:48,  1.10it/s]
8it [00:00, 17.94it/s][00:07<00:44,  1.18it/s]
8it [00:00, 19.97it/s][00:08<00:42,  1.21it/s]
8it [00:00, 17.84it/s] [00:08<00:40,  1.26it/s]
8it [00:00, 17.99it/s] [00:09<00:39,  1.27it/s]
7it [00:00, 19.83it/s] [00:10<00:38,  1.27it/s]
7it [00:00, 15.18it/s] [00:11<00:35,  1.34it/s]
7it [00:00, 19.39it/s] [00:11<00:35,  1.33it/s]
7it [00:00, 18.38it/s] [00:12<00:34,  1.35it/s]
7it [00:00, 19.37it/s] [00:13<00:32,  1.37it/s]
7it [00:00, 20.00it/s] [00:14<00:30,  1.42it/s]
7it [00:00, 17.90it/s] [00:14<00:29,  1.46it/s]
7it [00:00, 20.09it/s] [00:15<00:28,  1.45it/s]
7it [00:00, 18.57it/s] [00:16<00:28,  1.45it/s]
7it [00:00

In [11]:
a = await db.permutationsimulation.find_first(
    where={"block_number": block_number, "pool_address": pool_address, "n_permutations": n_permutations},
)

# Change the object and store back to database
a.data_location = new_blob_client.url
await a.


AttributeError: 'PermutationSimulation' object has no attribute 'save'